# Probabilistic CKY

### Learning Goals

Our goals today are to be able to...

1. Implement CKY parsing over *probabilistic* context free grammars
2. Describe how PCFGs can be thought of as language models.

### Getting started

Fortunately, today our goal is simply to extend our understanding of context-free grammars to include a notion of probability. In order to do this activity, you'll need to have completed the CKY activity from last time, so if you haven't, take time to complete, at minimum, your `CKYRecognizer` function. If you've given writing CKY a fair attempt and could use a reference implementation to check your understanding, you may study the implementation provided below.

As you saw in the reading and lecture, adding probabilities to CFGs can be motivated through *structural ambiguity*: When we generate from our grammar --- as you saw two activities ago! --- we sometimes have to make choices about which rule to apply. In the CFG activity, I told you to choose a rule *uniformly at random*. This is an assignment of probabilities to CFG generations! All we're doing now is allowing those to vary, and updating our parser to compute the probabilities we associate with each parse!

## A Probability-Augmented Grammar

### Today's goals

1. Below is a new grammar class describing PCNFGs (Probabilistic Chomsky-Normal Form Grammars). Take a moment to look through the provided methods (including the new `verify` function that is used to check whether the conditional probabilities sum to 1) and make sure you understand how things are represented. Note that the probabilities are supplied to the model as *raw* probabilities. Remember that once you start doing computations, you should make sure they're done as *log-probabilities!*



In [ ]:
from typing import Iterable, Sequence, Mapping, Tuple
import numpy as np
import random

class PCNFGrammar:
    def __init__(self,
                 S : str, 
                 T : Iterable[str], 
                 NT : Iterable[str], 
                 LR : Mapping[str, Sequence[str]],
                 GR : Mapping[str, Sequence[Sequence[str]]]):
        self.S = S
        self.T = T
        self.NT = NT
        self.LR = LR
        self.GR = GR

        # Ensure that the PCFG defines a probability distribution.
        self.verify()

    def get_start(self) -> str:
        return self.S
        
    def print_rules(self) -> None:
        print("---Lexical Rules")
        for lhs, rhss in self.LR.items():
            for rhs, p in rhss:
                print("{} -> {} ({})".format(lhs, rhs, p))
        print("---Grammar Rules")
        for lhs, rhss in self.GR.items():
            for rhs, p in rhss:
                print("{} -> {} ({})".format(lhs, " ".join(rhs), p))

    def verify(self, eps= 0.0001):
        for nt in NT:
            total = 0.0
            print("---{}".format(nt))
            for rhs,p in self.LR.get(nt, []):
                    print("{} -> {} ({})".format(nt, rhs, p))
                    total += p
            for rhs,p in self.GR.get(nt, []):
                    print("{} -> {} ({})".format(nt, " ".join(rhs), p))
                    total += p
            print("Sums to: {}".format(total))
            assert (total - 1.0)**2 < eps**2
                    
    def merges_to(self, B : str, C : str) -> Iterable[str]:
        # TODO: return a list of all NTs A with their probabilities p
        # such that A -> B C is in the grammar with probability p
        out = []
        for lhs, rhss in self.GR.items():
            for rhs, p in rhss:
                if (rhs[0] == B) and (rhs[1] == C):
                    out.append((lhs, np.log(p)))
        return out

    def tag(self, a : str) -> Sequence[str]:
        # TODO: return a list of all NTs A with probabilities p such that
        # A -> a is in the grammar with probability p
        out = []
        for lhs, rhss in self.LR.items():
            for rhs, p in rhss:
                if rhs == a:
                    out.append((lhs, np.log(p)))
        return out

In [ ]:
# From the J&M Reading

S = "S"
T = set(["book", "flight", "meal", "money", "include",
     "prefer", "does",
     "me", "I", "she",
     "Houston", "NWA",
     "the", "a", "this", "that",
     "from", "to", "on", "near", "through"])
NT = set(["N", "V", "X1", "X2", "D", "P", "Aux", "S", "NP", "Nom", "VP", "PP"])
LR = {
        # Lexical Rules
        "N": [("book", 0.1), 
              ("flight", 0.4), 
              ("meal", 0.05), 
              ("money", 0.05),
              ("dinner", 0.1),
              ("trip", 0.3)],
        "V": [("book", 0.3), 
              ("include", 0.3), 
              ("prefer", 0.4)],
        "D": [("the", 0.6), 
              ("a", 0.3),
              ("that", 0.1)],
        "P": [("from", 0.3), 
              ("to", 0.3), 
              ("on", 0.2), 
              ("near", 0.15), 
              ("through", 0.05)],
        "Aux": [("does", 0.6),
                ("can", 0.4)],
        "NP":[("you", 0.14), # Shares probability with Grammar Rules
              ("she", 0.0175), 
              ("I", 0.14), 
              ("me", 0.0525),
              ("Houston", 0.18), 
              ("NWA", 0.12),
              ("book", 0.1 * 0.75 * 0.15), 
              ("flight", 0.4 * 0.75 * 0.15), 
              ("meal", 0.05 * 0.75 * 0.15), 
              ("money", 0.05 * 0.75 * 0.15),
              ("dinner", 0.1 * 0.75 * 0.15),
              ("trip", 0.3 * 0.75 * 0.15)],
        "Nom":[("book", 0.075), 
              ("flight", 0.3), 
              ("meal", 0.0375), 
              ("money", 0.0375),
              ("dinner", 0.075),
              ("trip", 0.225)],
        "VP":[("book", 0.105), 
              ("include", 0.105), 
              ("prefer", 0.14)],
        "S":[("book", 0.00525), 
             ("include", 0.00525), 
             ("prefer", 0.007)]
}
GR = {
        # Grammar Rules
        "S":[(["NP", "VP"], 0.8),
             (["X1", "VP"], 0.15),
             (["V", "NP"], 0.01),
             (["X2", "PP"], 0.0075),
             (["V", "PP"], 0.0075),
             (["VP", "PP"], 0.0075)],
        "X1":[(["Aux", "NP"], 1.0)],
        "NP":[(["D", "Nom"], 0.2),
              (["Nom", "N"], 0.03),
              (["Nom", "PP"], 0.0075)],
        "Nom":[(["Nom", "N"], 0.2), 
               (["Nom", "PP"], 0.05)],
        "VP":[(["V", "NP"], 0.2), 
              (["X2", "PP"], 0.15),
              (["V", "PP"], 0.15),
              (["VP", "PP"], 0.15)],
        "X2":[(["V", "NP"], 1.0)],
        "PP":[(["P", "NP"], 1.0)]
}

JMGrammar = PCNFGrammar(S, T, NT, LR, GR)
JMGrammar.verify()

In [ ]:
print(JMGrammar.merges_to("D", "Nom")) # Should be [NP] w/ probabilities
print(JMGrammar.tag("book")) # Should be [N, V, Nom, VP, S] w/ probabilities

## Reviewing CKY Recognition

Below is an implementation of a CKY *Recognizer* from the previous activity, but adapted to work with the PCNFGrammar defined above. However, it ignores the probabilities! 

1. **Check the implementation below against your's**: Make sure you understand how the implementation works. Take a moment to reflect: Is the implementation simpler or trickier than you expected? Was your solution (or partial solution) missing coverage of some situations (or is my implementation? I accept code reviews!)?

In [ ]:
def printChart(chart : Sequence[Sequence[Sequence[str]]]) -> None:
    # May be useful for debugging your chart
    for i in range(len(chart)):
        box_len = max([len(chart[i][j]) 
                       for j in range(len(chart[i]))])
        for k in range(box_len):
            print("\t|".join(("" if k >= len(x) else 
                              "({}, {:.2})".format(*x[k])
                             ).ljust(20) 
                            for x in chart[i])
                 )
        print()

In [ ]:
def CKYRecognizer(grammar : PCNFGrammar, sent : Sequence[str]) -> bool:
    # chart[i][j] contains the NTs that correspond to the span from i to j+1
    # (i.e., sent[i:j+1])
    chart = [[[] for _ in range(len(sent))] for _ in range(len(sent))]

    # Tag each word with potential NTs
    for i, w in enumerate(sent):
        chart[i][i] += grammar.tag(w)

    # Compute merges.
    # width = span length
    for width in range(len(sent)):
        # i is the left index
        for i in range(len(sent) - width):
            # j is the right index
            j = i + width
            # k is the split position
            for k in range(i, j):
                # check left half's possible NTs
                for B in chart[i][k]:
                    # right half's possible NTs
                    for C in chart[k+1][j]:
                        # Check grammar rules for possible LHSs
                        chart[i][j] += grammar.merges_to(B[0], C[0])
                # Uncomment to monitor each cell update 
                # printChart(chart)
                # print("----")
    # If we have the start symbol in the cell corresponding to the full span, 
    # sent is in the grammar!
    for nt, p in chart[0][-1]:
        if grammar.get_start() == nt:
            return True
    return False

In [ ]:
print(CKYRecognizer(JMGrammar, "book the flight through Houston".split())) # Should be grammatical
print(CKYRecognizer(JMGrammar, "Houston the book flight through".split())) # Should be ungrammatical

## Implementing Probabilistic CKY

2. Modify the `CKYRecognizer` implementation so that it returns the *probability* of the start symbol deriving that sentence rather than whether it could be generated. 

In [ ]:
def PCKYRecognizer(grammar : CNFGrammar, sent : Sequence[str]) -> float:
    # TODO: Implement the probabilistic CKY algorithm. Borrow code from the CKYRecognizer and modify it!
    return False